<a href="https://colab.research.google.com/github/Vivek-1116/SDAE-and-VAE-for-Cancer-Classification-through-Multi-omics-Feature-Extraction/blob/main/SDAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Stacked Denoising Autoencoder Model (SDAE)
This model includes Feature Selection (FS) technique "Recursive Feature Elimination (RFE)" and Class Imbalance algorithm "Synthetic Minority Oversampling Technique (SMOTE)" to further reduce data complexity & enhance computational efficiency.

IMPORT LIBRARIES

In [ ]:
import numpy as np # Lib for multi-dimensional arrays and matrices handling
import pandas as pd # Lib for data manipulation and analysis

from sklearn import svm # SVM model for RFE Feature Selection
from sklearn.preprocessing import * # Data pre-processing and standardization
from sklearn.preprocessing import MinMaxScaler # Feature scaling
from sklearn.pipeline import Pipeline # To fit hyperparameters into 1 pipeline
from sklearn.model_selection import train_test_split # Splits data into indices of training and testing
from imblearn.over_sampling import SMOTE # Oversample data using SMOTE algorithm

import warnings # Lib for warning issue handling
warnings.filterwarnings('ignore') # Ignores all irrelevant warnings
from collections import Counter # To get / set count of elements

%matplotlib inline
import matplotlib.pyplot as plt # Lib for interactive plots
plt.style.use('seaborn-white') # Sets theme of visualization (seaborn-ticks / whitegrid) are similar to white
import seaborn as sns # Matplotlib based lib - better interface for drawing attractive and informative statistical graphics
sns.set_palette(['#FC4B60','#06B1F0']) 
random_seed = 63445 

# Framework / Platform for building ML models
import tensorflow 
from tensorflow.keras import regularizers 
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Model, Sequential, load_model, save_model
from tensorflow.keras.layers import Input, Dense, Activation, GaussianNoise, LeakyReLU, Dropout

print("TensorFlow Version:",tensorflow.__version__)

MOUNT DRIVE & SET DATA PATH

In [ ]:
from google.colab import drive # Link notebook with google drive
drive.mount('/content/gdrive/') # To retrieve data from our personal Gdrive (can remove this line if we are about to access data from pc)
# Define path
data_path = '/content/gdrive/My Drive/PSM2 VIVEK/LUSC Dataset/' # Change this path accordingly

CHECKING FOR MACHINE RUNTIME TYPE

In [ ]:
# Checking whether your machine running only on CPU or with GPU
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
CUDA_VISIBLE_DEVICES = 1

IMPORT DATASET

In [ ]:
# Create DataFrame
print("Reading Multi-omics Data")
df = pd.read_csv(data_path + "Complete_MultiOmics.csv", delimiter=",", index_col=0)
nFeatures = len(df.columns) - 2 # First and last column does not hold data to be processed
print("Number of features :", nFeatures)
print("Size of Dataset :", df.shape)

DISPLAY MULTI-OMICS DATA

In [ ]:
print("Multi-omics data imported successfully")
df

# RECURSIVE FEATURE ELIMINATION (RFE)

DATA AGGREGATION (FEATURES & TARGETS)

In [ ]:
features = df.iloc[:,1:-1] #Retrieves all rows (1:), leaves last column (,1:-1)
target = df.iloc[:,-1] #Retrieves all rows (1:), retrieves only last column (,-1)

print("Number of features :", features.shape[1])
print("Number of targets :", target.shape[0])

DATA STANDARDIZATION

In [ ]:
#Setting all dataset into a range of 0 to 1
min_max_scaler = MinMaxScaler(feature_range =(0, 1)) 
# Scaled feature 
features = pd.DataFrame(min_max_scaler.fit_transform(features)) 

SVM MODEL FITTING

In [ ]:
#Function to Define SVM model accuracy
def accuracy(model, features, target):
	prediction = model.predict(features)
	print ("Accuracy of model:", accuracy_score(target, prediction) * 100, "%")
    
warnings.filterwarnings("ignore")

In [ ]:
#Set Parameter
C = 1.0
rfeIndex = nFeatures
#Create SVM model using a linear kernel
model = svm.SVC(kernel='linear', C=C).fit(features,target)
coef = model.coef_

#Print co-efficients of features
for i in range(0, nFeatures):
	print (features.columns[i],":", coef[0][i])

RFE IMPLEMENTATION

In [ ]:
#Find the minimum weight among features and eliminate the feature with the smallest weight
min = coef[0][0]

for j in range(6663): # To make sure only 12,000 features remain
	index = 0
	for i in range(0,rfeIndex): # Iterates until the final feature
		if min > coef[0][i]:
			index = index + 1
			min = coef[0][i]

	if len(features.columns) == 1:
		print ("After recursive elimination we have the", features.columns[index], "feature with a score of:", min)
  
	else:
		print ("Lowest feature weight is for", features.columns[index], "with a value of:", min)
		print ("Dropping feature", features.columns[index])  
		features.drop(features.columns[index], axis = 1, inplace = True)
		rfeIndex = rfeIndex - 1
		nFeatures = nFeatures - 1

FEATURES AFTER RFE

In [ ]:
print("Dataset size after Feature Selection:",features.shape)

NORMALIZED FEATURES

In [ ]:
features

# SYNTHETIC MINORITY OVERSAMPLING TECHNIQUE (SMOTE)

CLASS IMBALANCE COUNTER

In [ ]:
from collections import Counter
counter = Counter(df['Class'])
counter

CLASS IMBALANCE PLOT (VISUALIZATION)

In [ ]:
sns.set(style="darkgrid")
ax = sns.countplot(df['Class'])
ax.set(xlabel='Class', ylabel='Frequency')
plt.show()

DATA SPLITTING

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size = 0.25, random_state=random_seed, stratify=target)
print ("X TRAIN DATA SHAPE: ", x_train.shape)
print ("X TEST DATA SHAPE: ", x_test.shape)
print ("Y TRAIN DATA SHAPE: ", y_train.shape)
print ("Y TEST DATA SHAPE: ", y_test.shape)

SMOTE MODEL FITTING

In [ ]:
sm = SMOTE(k_neighbors=1, random_state=random_seed)
X, Y = sm.fit_sample(x_train, y_train)
print ('Shape of oversampled data: {}'.format(X.shape))
print ('Shape of Y: {}'.format(Y.shape))

BALANCED DATA VISUALIZATION

In [ ]:
sns.set(style="darkgrid")
ax = sns.countplot(Y)
ax.set(xlabel='Class', ylabel='Frequency')
plt.title('Balanced training data')
plt.show()

CLASS (TARGET) AFTER SMOTE

In [ ]:
print('Resampled dataset shape for Train {}'.format(Counter(Y)))
print('Normal validation dataset shape for Test {}'.format(Counter(y_test)))

# AUTOENCODER MODEL

PARAMETER SETTING FOR DAE MODEL TRAINING

In [ ]:
# Stop training model when "monitor parameter" has stopped improving
earlyStopping = EarlyStopping(monitor='loss', patience=50) # patience is num of epochs to reach early stopping
# Save model after every epoch
checkpointer = ModelCheckpoint(filepath='MO_SDAE_Training.h5', verbose=1, save_best_only=True)
input_dims = (len(features)-1)

DENOISING AUTOENCODER (DAE)

In [ ]:
def dae (inputX, input_dims, output_dims, epoch, activation, loss, opti):
            
    model = Sequential()
    
    if input_dims > 9999:
        with tensorflow.device('/cpu:0'):
            print("Using CPU....")
            model.add(Dense(input_dims, input_dim = input_dims))
            model.add(GaussianNoise(0.5)) 
            model.add(Dense(output_dims, activation = activation, kernel_regularizer = regularizers.l1(0.01)))
            model.add(Dense(input_dims, activation= activation))
            model.compile(loss = loss, optimizer = opti)
            model.fit(inputX, inputX, epochs = epoch, batch_size = 4)
            model.summary()
    else:
        with tensorflow.device('/device:GPU:0'):
            print("Using GPU....")
            model.add(Dense(input_dims, input_dim = input_dims))
            model.add(GaussianNoise(0.5)) 
            model.add(Dense(output_dims, activation= activation, kernel_regularizer = regularizers.l1(0.01)))
            model.add(Dense(input_dims, activation= activation))
            model.compile(loss = loss, optimizer = opti)
            model.fit(inputX, inputX, epochs = epoch, batch_size = 4)
            model.summary()
    
    return model

In [ ]:
autoencoder = dae(X, 
                  input_dims = 12000,
                  output_dims = 500,
                  epoch = 5,
                  activation = 'relu', 
                  loss = 'mse',
                  opti = 'adam', 
                 )

HYPER-PARAMETER SETTING

In [ ]:
# Hyper-parameter
layers = [12000, 10000, 8000, 6000, 4000, 2000, 500] # Setting the size of your layer 
epoch = 3 
optimizer = 'adam'
activation = 'relu'
loss = 'mse'

STACKED DENOISING AUTOENCODER (SDAE) - PRETRAIN MODEL

In [ ]:
def sdae_pretrain (inputX, layers, activation, epoch, optimizer, loss):
    
    encoder = []
    decoder = []
    ae = []
    
    for i in range(len(layers)-1):
            print("Now pretraining layer {} [{}-->{}]".format(i+1, layers[i], layers[i+1]))

            input_dims = layers[i]
            output_dims = layers[i+1]
            
            autoencoder = dae(inputX, input_dims, output_dims, epoch, activation, loss, optimizer)
            enc = Sequential()
            enc.add(Dense(output_dims, input_dim=input_dims))
            enc.compile(loss=loss, optimizer=optimizer) 
            enc.layers[0].set_weights(autoencoder.layers[2].get_weights())
            inputX = enc.predict(inputX)
            print("check dimension : ", inputX.shape)
            enc.summary()
            encoder.append(autoencoder.layers[2].get_weights())
            decoder.append(autoencoder.layers[3].get_weights())
            ae.append(autoencoder)
    
    return ae

In [ ]:
Train_SDAE = sdae_pretrain( X, layers = layers, activation = activation, epoch = epoch, optimizer = optimizer, loss = loss)

STORING PRE-TRAINED MODEL

In [ ]:
for i, m in enumerate(Train_SDAE):
    filename="pretrain_model" + str(i) + ".hd5"
    m.save(filename)

LOADING PRE-TRAINED MODEL

In [ ]:
Train_SDAE = []
with tensorflow.device('/cpu:0'):
    for i in range (len(layers)-1):
      Train_SDAE.append(load_model("pretrain_model"+ str(i) + ".hd5"))

DEFINING CONFUSION MATRIX

In [ ]:
from tensorflow.keras import backend as K

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1(y_true, y_pred):
    precision2 = precision(y_true, y_pred)
    recall2 = recall(y_true, y_pred)
    return 2*((precision2 * recall2)/(precision2 + recall2 + K.epsilon()))

STACKED DENOISING AUTOENCODER - FINE TUNING

In [ ]:
def fine_tuning(weights, inputX, inputXt, inputY, inputYt, layers, epoch, batch, optimizer, loss):

    encoder = []
    decoder = []

    for i in range(len(Train_SDAE)):
    
        encoder.append(Train_SDAE[i].layers[2].get_weights())
        decoder.append(Train_SDAE[i].layers[3].get_weights())
    
    with tensorflow.device('/device:gpu:0'): #I have to put this because the model size is too big for my GPU
        ft = Sequential()
        ft.add(Dense(layers[0], input_dim=layers[0]))
        ft.add(GaussianNoise(0.5))

        for i in range(len(layers)-1):
            ft.add(Dense(layers[i+1], activation = 'relu', weights = encoder[i], kernel_regularizer = regularizers.l2(0.01))) # Initial regularizer (l1_l2)
        
        for i in reversed(range(len(layers)-1)):
            ft.add(Dense(layers[i], activation = 'relu', weights = decoder[i]))
    ft.add(Dropout(0.2))
    ft.add(Dense(200, activation = 'relu'))
    ft.add(Dense(150, activation = 'relu', use_bias=True))
    ft.add(Dense(100, activation = 'relu', kernel_initializer= "glorot_uniform", bias_initializer= "zeros")) 
    ft.add(Dense(50, activation= 'relu', kernel_initializer= "glorot_uniform", bias_initializer= "zeros"))   
    ft.add(Dense(1, activation = 'sigmoid')) 
    
    ft.compile(loss=loss, optimizer=optimizer, metrics=['accuracy', recall, precision, f1])
    History = ft.fit(X, Y, epochs = epoch, batch_size = batch, validation_data = (x_test, y_test))
    ft.summary()

    plt.plot(History.history['loss']) 
    plt.plot(History.history['val_loss']) 
    plt.title('SDAE Model Loss') 
    plt.ylabel('Loss') 
    plt.xlabel('Epoch') 
    plt.legend(['Train', 'Test'], loc='upper right') 
    plt.show()

    return ft

In [ ]:
Result = fine_tuning(Train_SDAE, X, x_test, Y, y_test, layers = layers, epoch = 50, batch = 4, optimizer = 'adam', loss = 'binary_crossentropy') 